In [ ]:
import torch
from dataloader import data_loader
import torch.nn as nn
from torchsummary import summary
from ResNet import ResNet
from train import train
from evaluate import evaluate

In [ ]:
train_loader, valid_loader = data_loader(
    data_dir=r"./data",
    batch_size=64,
    num_workers=4,
    pin_memory=True,
)
test_loader = data_loader(
    data_dir=r"./data",
    batch_size=64,
    test=True,
    num_workers=4,
    pin_memory=True,
)

In [ ]:
out_neurons = 1
num_epochs = 100
batch_size = 16
learning_rate = 0.001
patience = 5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = ResNet(layers=[3, 4, 6, 3]).to(device)
model = ResNet(layers=[2, 2, 2, 2], out_neurons=out_neurons).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=learning_rate,
    weight_decay=0.001,
    momentum=0.9,
)

total_step = len(train_loader)

summary(
    model,
    input_size=(3, 224, 224),
    device="cuda" if torch.cuda.is_available() else "cpu",
)

In [ ]:
train(
    model=model,
    device=device,
    num_epochs=num_epochs,
    train_loader=train_loader,
    criterion=criterion,
    optimizer=optimizer,
    valid_loader=valid_loader,
    patience=patience,
)

In [ ]:
best_model = ResNet(layers=[2, 2, 2, 2]).to(device)
best_model.load_state_dict(torch.load("best_model.pth", weights_only=True))

In [ ]:
evaluate(
    model=model,
    test_loader=test_loader,
    device=device,
)